In [ ]:
from fastai import *          # Quick accesss to most common functionality
from fastai.tabular import *  # Quick accesss to tabular functionality     # Access to example data provided with fastai

# Tabular example

Tabular data should be in a Pandas `DataFrame`.

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [ ]:
dep_var = '>=50k'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [ ]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [ ]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test, label=0)
                           .databunch())

In [ ]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
Private,HS-grad,Never-married,Other-service,Not-in-family,White,False,-0.9959,-0.6220,-0.4224
Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Asian-Pac-Islander,False,-0.1896,0.7520,-0.4224
Self-emp-not-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,-0.4095,0.2697,-0.4224
Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,False,0.3968,-0.4025,-0.0312
Self-emp-inc,Bachelors,Never-married,Sales,Not-in-family,White,False,-0.7760,-1.4471,1.1422
Local-gov,Masters,Never-married,Prof-specialty,Not-in-family,White,False,0.9831,0.4668,1.5334
Self-emp-not-inc,Doctorate,Married-civ-spouse,Prof-specialty,Husband,White,False,2.0826,0.2671,2.3157
Private,Some-college,Never-married,Craft-repair,Own-child,Amer-Indian-Eskimo,False,-0.7760,-1.5526,-0.0312
State-gov,Prof-school,Married-civ-spouse,Prof-specialty,Wife,White,False,-0.1896,5.2903,1.9245
Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,False,0.6899,-1.0076,-0.4224


In [ ]:
learn = get_tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit(1, 1e-2)

Total time: 00:03
epoch  train_loss  valid_loss  accuracy
1      0.360272    0.385143    0.820000  (00:03)



In [ ]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50k
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,1
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,1
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,0
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,1
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,0


In [ ]:
row = df.iloc[0]

In [ ]:
learn.predict(row)

(1, tensor(0), tensor([0.5647, 0.4353]))